# k-Anonymity

In this notebook, we'll explore data anonymization, in particular "k-anonymity" [1]. We will implement a simple algorithm [2] to produce a k-anonymous dataset.

For more reading on the topic, please see: 

- [k-Anonymity: A Model For Protecting Privacy. Latanya Sweeney](https://epic.org/privacy/reidentification/Sweeney_Article.pdf)
- [Mondrian - Multidimensional k-Anonymity](https://www.utdallas.edu/~muratk/courses/privacy08f_files/MultiDim.pdf)

## Introduction


k-anonymity protects the privacy of individual persons by pooling their attributes into groups of at least $k$ people. The method is based on the assumption that we have a dataset that contains $N$ entries. Each entry consists of a list of $D$ attributes $X_i$ ($i \in [0,D]$) that contain (non-sensitive) information about a person, such as age, gender, zip code of residence, etc. These attributes are called "quasi-identifiers", as combining several of them into a "super-identifier" can often uniquely identify a person even in large datasets (e.g. the combination of gender, age and zip code might be so specific that only a single person in a dataset has a given combination). In addition, the model assumes that the dataset contains a single sensitive attribute that contains e.g. information about a person's income and that we want to protect. The method can also be generalized to datasets with more than one sensitive attribute or datasets where there's no clear distinction between quasi-identifiers and sensitive attributes. For this case study we will look at the simple case though.

Now, k-anonymity demands that we group individual rows/persons of our dataset into group of at least $k$ rows/persons and replace the quasi-identifier attributes of these rows with aggregate quantities, such that it is no longer possible to read the individual values. This protects people by ensuring that an adversary who knows all values of a person's quasi-identifier attributes can only find out which group a person might belong to but not know if the person is really in the dataset.

A large problem of this approach is that it might happen that all people in a k-anonymous group possess the same value of the sensitive attribute. An adversary who knows that a person is in that k-anonymous group can then still learn the value of the sensitive attribute of that person with absolute certainty. This problem can be fixed by using an extension of k-anonymity called "l-diversity": l-diversity ensures that each k-anonymous group contains at least l different values of the sensitive attribute. Therefore, evne if an adversary can identify the group of a person he/she still would not be able to find out the value of that person's sensitive attribute with certainty. 

However, even when using l-diversity an adversary could still learn some information about a person's sensitive attribute using probabilistic reasoning: If, for example, 4 out of 5 people in a 5-anonymous group possess a given value of the sensitive attribute, an attacker can reason that a given person who he/she knows is part of the group will -with high probability- possess that value. Again, this problem can be fixed by extending k-anonymity using a so-called "t-closeness" criterion: t-closeness demands that the statistical distribution of the sensitive attribute values in each k-anonymous group is "close" to the overall distribution of that attribute in the entire dataset. Typically, the closeness can be measured using e.g. the Kullback-Leibler (KL) divergence. An adversary could then only learn a limited amount of information from comparing the distribution of the values in the group to the distribution in the entire dataset.

Of course, k-anonymity, l-diversity and t-closeness all limit the amount of information that a legitimate user can learn from the data as well, so typically we need to balance the degree of privacy against the utility of the resulting data.

## Implementing k-anonymity

Turning a dataset into a k-anonymous (and possibly l-diverse or t-close) dataset is a complex problem, and finding the optimal partition into k-anonymous groups is an NP-hard problem. Fortunately, several practical algorithms exists that often produce "good enough" results by employing greedy search techniques.

In this tutorial we will explore the so-called "Mondrian" algorithm (see link above), which uses a greedy search algorithm to parition the original data into smaller and smaller groups (if we plot the resulting partition boundaries in 2D they resemble the pictures by Piet Mondrian, hence the name).

The algorithm assumes that we have converted all attributes into numerical or categorical values and that we're able to measure the "span" of a given data attribute $X_i$ (we'll explain that in more detail later).

### Partitioning

The algorithm proceeds then as follows to partition the data into k-anonymous gorups:

1. Initialize the finished set of partitions to an empty set $P_{finished} = \{\}$.
2. Initialize the working set of paritions to a set containing a partition with the entire dataset $P_{working} = \{\{1, 2,\dots ,N\}\}$.
4. While there are partitions in the working set, pop one partition from it and
  * Calculate the relative spans of all columns in the partition.
  * Sort the resulting columns by their span (in descending order) and iterate over them. For each column,
      * Try to split the partition along that column using the median of the column values as the split point.
      * Check if the resulting partitions are valid according to our k-anonymity (and possibly additional) criteria.
      * If yes, add the two new partitions to the working set and break out of the loop.
  * If no column produced a valid split, add the original partition to the set of finished partitions.
5. Return the finished set of partitions

### Data Aggregation

After obtaining the partitions we still need to aggregate the values of the quasi identifiers and the sensitive attributes in each k-anonymous group. For this, we can e.g. replace numerical attributes with their range (e.g. "age: 24-28") and categorical attributes with their union (e.g. "employment-group: [self-employed, employee, worker]"), though other aggregations are possible. Methods like [Anatomy](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.156.9150&rep=rep1&type=pdf) even preserve the micro-data in each group, which increases re-identification risk though.

## Usage of the Anonymized Data

We can then use the anonymized data for privacy-preserving machine learning, e.g. using scikit-learn. If we have time at the end of this part we'll cover this as well. With the dataset generation method discussed here you can use the resulting datasets just as you would use a normal dataset though, so there's really nothing special about the anonymized data here.

Let's get started with the anonymization!

In [1]:
# we use Pandas to work with the data as it makes working with categorical data very easy
import pandas as pd

import matplotlib.pylab as pl
import matplotlib.patches as patches

#Tkinter GUI
import tkinter as tk
import tkinter.ttk as ttk
from tkinter import messagebox
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import csv
from tkinter import VERTICAL, HORIZONTAL
from tkscrolledframe import ScrolledFrame

In [2]:
# this is a list of the column names in our dataset (as the file doesn't contain any headers)
names = (
    'age',
    'workclass', #Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
    'fnlwgt', # "weight" of that person in the dataset (i.e. how many people does that person represent) -> https://www.kansascityfed.org/research/datamuseum/cps/coreinfo/keyconcepts/weights
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income',
)

# some fields are categorical and will require special treatment
categorical = set((
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'sex',
    'native-country',
    'race',
    'income',
))
df = pd.read_csv("data/k-anonymity/adult.all.txt", sep=", ", header=None, names=names, index_col=False, engine='python');# We load the data using Pandas

In [3]:
#df.head()

In [4]:
for name in categorical:
    df[name] = df[name].astype('category')

**Implement a function that returns the spans (max-min for numerical columns, number of different values for categorical columns) of all columns for a partition of a dataframe.** 

In [5]:
def get_spans(df, partition, scale=None):
    """
    :param        df: the dataframe for which to calculate the spans
    :param partition: the partition for which to calculate the spans
    :param     scale: if given, the spans of each column will be divided
                      by the value in `scale` for that column
    :        returns: The spans of all columns in the partition
    """
    spans = {}
    for column in df.columns:
        if column in categorical:
            span = len(df[column][partition].unique())
        else:
            span = df[column][partition].max()-df[column][partition].min()
        if scale is not None:
            span = span/scale[column]
        spans[column] = span
    return spans

In [6]:
full_spans = get_spans(df, df.index)
full_spans

{'age': 73,
 'workclass': 9,
 'fnlwgt': 1478115,
 'education': 16,
 'education-num': 15,
 'marital-status': 7,
 'occupation': 15,
 'relationship': 6,
 'race': 5,
 'sex': 2,
 'capital-gain': 99999,
 'capital-loss': 4356,
 'hours-per-week': 98,
 'native-country': 42,
 'income': 2}

**Implement a `split` function that takes a dataframe, partition and column and returns two partitions that split the given partition such that all rows with values of the column `column` below the median are in one partition and all rows with values above or equal to the median are in the other.**

In [7]:
def split(df, partition, column):
    """
    :param        df: The dataframe to split
    :param partition: The partition to split
    :param    column: The column along which to split
    :        returns: A tuple containing a split of the original partition
    """
    dfp = df[column][partition]
    if column in categorical:
        values = dfp.unique()
        lv = set(values[:len(values)//2])
        rv = set(values[len(values)//2:])
        return dfp.index[dfp.isin(lv)], dfp.index[dfp.isin(rv)]
    else:        
        median = dfp.median()
        dfl = dfp.index[dfp < median]
        dfr = dfp.index[dfp >= median]
        return (dfl, dfr)

Now that we have all helper functions in place, we can implement the partition algorithm discussed above:

**Implement the partitioning algorithm discussed above, using a k-anonymous criterion for the partitions you create.**

In [8]:
def is_k_anonymous(df, partition, sensitive_column, k=3):
    """
    :param               df: The dataframe on which to check the partition.
    :param        partition: The partition of the dataframe to check.
    :param sensitive_column: The name of the sensitive column
    :param                k: The desired k
    :returns               : True if the partition is valid according to our k-anonymity criteria, False otherwise.
    """
    if len(partition) < k:
        return False
    return True

def partition_dataset(df, feature_columns, sensitive_column, scale, is_valid):
    """
    :param               df: The dataframe to be partitioned.
    :param  feature_columns: A list of column names along which to partition the dataset.
    :param sensitive_column: The name of the sensitive column (to be passed on to the `is_valid` function)
    :param            scale: The column spans as generated before.
    :param         is_valid: A function that takes a dataframe and a partition and returns True if the partition is valid.
    :returns               : A list of valid partitions that cover the entire dataframe.
    """
    finished_partitions = []
    partitions = [df.index]
    while partitions:
        partition = partitions.pop(0)
        spans = get_spans(df[feature_columns], partition, scale)
        for column, span in sorted(spans.items(), key=lambda x:-x[1]):
            lp, rp = split(df, partition, column)
            if not is_valid(df, lp, sensitive_column) or not is_valid(df, rp, sensitive_column):
                continue
            partitions.extend((lp, rp))
            break
        else:
            finished_partitions.append(partition)
    return finished_partitions


Now let's try this on our dataset! To keep things simple, we will at first select only two columns from the dataset that we apply the partitioning to. This makes it easier to check/visualize the result and speed up the execution (the naive algorithm can take several minutes when running on the entire dataset) 

In [9]:
# we apply our partitioning method to two columns of our dataset, using "income" as the sensitive attribute
feature_columns = ['age', 'education-num']
sensitive_column = 'income'
finished_partitions = partition_dataset(df, feature_columns, sensitive_column, full_spans, is_k_anonymous)

In [10]:
# we get the number of partitions that were created
len(finished_partitions)

500

Let's visualize the created partitions! To do that, we will write functions to get the rectangular bounds of a partition along two columns. We can then plot these rects to see how our partitioning function divides the dataset. If we perform the partition only along the two columns selected for plotting then the resulting rects should not overlap and cover the entire dataset.

In [11]:
def build_indexes(df):
    indexes = {}
    for column in categorical:
        values = sorted(df[column].unique())
        indexes[column] = { x : y for x, y in zip(values, range(len(values)))}
    return indexes

def get_coords(df, column, partition, indexes, offset=0.1):
    if column in categorical:
        sv = df[column][partition].sort_values()
        l, r = indexes[column][sv[sv.index[0]]], indexes[column][sv[sv.index[-1]]]+1.0
    else:
        sv = df[column][partition].sort_values()
        next_value = sv[sv.index[-1]]
        larger_values = df[df[column] > next_value][column]
        if len(larger_values) > 0:
            next_value = larger_values.min()
        l = sv[sv.index[0]]
        r = next_value
    # we add some offset to make the partitions more easily visible
    l -= offset
    r += offset
    return l, r

def get_partition_rects(df, partitions, column_x, column_y, indexes, offsets=[0.1, 0.1]):
    rects = []
    for partition in partitions:
        xl, xr = get_coords(df, column_x, partition, indexes, offset=offsets[0])
        yl, yr = get_coords(df, column_y, partition, indexes, offset=offsets[1])
        rects.append(((xl, yl),(xr, yr)))
    return rects

def get_bounds(df, column, indexes, offset=1.0):
    if column in categorical:
        return 0-offset, len(indexes[column])+offset
    return df[column].min()-offset, df[column].max()+offset

In [12]:
# we calculate the bounding rects of all partitions that we created
indexes = build_indexes(df)
column_x, column_y = feature_columns[:2]
rects = get_partition_rects(df, finished_partitions, column_x, column_y, indexes, offsets=[0.0, 0.0])

In [13]:
# let's see how our rects look like
rects[:10]

[((17.0, 7.0), (18.0, 9.0)),
 ((18.0, 7.0), (20.0, 9.0)),
 ((21.0, 10.0), (22.0, 11.0)),
 ((25.0, 10.0), (27.0, 11.0)),
 ((37.0, 9.0), (39.0, 10.0)),
 ((37.0, 10.0), (38.0, 13.0)),
 ((41.0, 10.0), (43.0, 13.0)),
 ((39.0, 13.0), (41.0, 16.0)),
 ((46.0, 10.0), (48.0, 13.0)),
 ((46.0, 13.0), (48.0, 16.0))]

In [14]:
# we plot the rects
def plot_rects(df, ax, rects, column_x, column_y, edgecolor='black', facecolor='none'):
    for (xl, yl),(xr, yr) in rects:
        ax.add_patch(patches.Rectangle((xl,yl),xr-xl,yr-yl,linewidth=1,edgecolor=edgecolor,facecolor=facecolor, alpha=0.5))
    ax.set_xlim(*get_bounds(df, column_x, indexes))
    ax.set_ylim(*get_bounds(df, column_y, indexes))
    ax.set_xlabel(column_x)
    ax.set_ylabel(column_y)

In [15]:
#tkinter GUI - graphs
def DrawkAnonymity():
    figure1_window = tk.Tk()
    figure1_window.geometry('1200x700')
    figure1_window.title('K-ANONYMITY')

    figure1 = pl.figure(figsize=(17,10))
    ax = pl.subplot(111)

    graph1 = FigureCanvasTkAgg(figure1, figure1_window)
    graph1.get_tk_widget().grid(row=0,column=0)

    plot_rects(df, ax, rects, column_x, column_y, facecolor='r')
    pl.scatter(df[column_x], df[column_y])
    #plshow()
    
    figure1_window.mainloop()

# Generating an k-Anonymous Dataset

Of course, to use the data we want to produce a new dataset that contains one row for each partition and value of the sensitive attribute. To do this, we need to aggregate the columns in each partition.  Let's do this!

In [16]:
def agg_categorical_column(series):
    return [','.join(set(series))]

def agg_numerical_column(series):
    return [series.mean()]

In [17]:
def build_anonymized_dataset(df, partitions, feature_columns, sensitive_column, max_partitions=None):
    aggregations = {}
    for column in feature_columns:
        if column in categorical:
            aggregations[column] = agg_categorical_column
        else:
            aggregations[column] = agg_numerical_column
    rows = []
    for i, partition in enumerate(partitions):
        if i % 100 == 1:
            print("Finished {} partitions...".format(i))
        if max_partitions is not None and i > max_partitions:
            break
        grouped_columns = df.loc[partition].agg(aggregations, squeeze=False)
        sensitive_counts = df.loc[partition].groupby(sensitive_column).agg({sensitive_column : 'count'})
        values = grouped_columns.iloc[0].to_dict()
        for sensitive_value, count in sensitive_counts[sensitive_column].items():
            if count == 0:
                continue
            values.update({
                sensitive_column : sensitive_value,
                'count' : count,

            })
            rows.append(values.copy())
    return pd.DataFrame(rows)

In [18]:
dfn = build_anonymized_dataset(df, finished_partitions, feature_columns, sensitive_column)

Finished 1 partitions...
Finished 101 partitions...
Finished 201 partitions...
Finished 301 partitions...
Finished 401 partitions...


In [19]:
# we sort the resulting dataframe using the feature columns and the sensitive attribute
dfn.sort_values(feature_columns+[sensitive_column])

,age,count,education-num,income
469,17.000000,3,3.000000,<=50k
615,17.000000,5,4.000000,<=50k
110,17.000000,36,5.000000,<=50k
111,17.000000,198,6.000000,<=50k
0,17.000000,334,7.200599,<=50k
120,17.000000,14,9.000000,<=50k
43,17.000000,5,10.000000,<=50k
616,18.000000,6,4.000000,<=50k
329,18.000000,10,5.000000,<=50k
121,18.000000,249,9.000000,<=50k


# Implementing l-diversity (the naive way)

Now let's see how we can implement l-diversity in order to protect the privacy of the persons in the dataset even better. To implement l-diversity, we can do the following things:

* Modify our `is_valid` function to not only check for the size of a given partition but also ensure that the values of the sensitive attribute in the partition are diverse enough.
* Modify the `split` function to produce splits that are diverse (if possible)

**Implement a validator function that returns `True` if a given partition contains at least `l` different values of the sensitive attribute, `False` otherwise.**

In [20]:
def diversity(df, partition, column):
    return len(df[column][partition].unique())

def is_l_diverse(df, partition, sensitive_column, l=2):
    """
    :param               df: The dataframe for which to check l-diversity
    :param        partition: The partition of the dataframe on which to check l-diversity
    :param sensitive_column: The name of the sensitive column
    :param                l: The minimum required diversity of sensitive attribute values in the partition
    """
    return diversity(df, partition, sensitive_column) >= l

In [21]:
# now let's apply this method to our data and see how the result changes
finished_l_diverse_partitions = partition_dataset(df, feature_columns, sensitive_column, full_spans, lambda *args: is_k_anonymous(*args) and is_l_diverse(*args))

In [22]:
len(finished_l_diverse_partitions)

309

In [23]:
column_x, column_y = feature_columns[:2]
l_diverse_rects = get_partition_rects(df, finished_l_diverse_partitions, column_x, column_y, indexes, offsets=[0.0, 0.0])

In [24]:
def DrawlDiversity():
    figure2_window = tk.Tk()
    figure2_window.geometry('1200x700')
    figure1_window.title('L-DIVERSITY')
    
    figure2 = pl.figure(figsize=(17,10))
    ax = pl.subplot(111)
    
    graph2 = FigureCanvasTkAgg(figure2, figure2_window)
    graph2.get_tk_widget().pack(side=tk.LEFT, fill = tk.BOTH)
    
    plot_rects(df, ax, l_diverse_rects, column_x, column_y, edgecolor='b', facecolor='b')
    plot_rects(df, ax, rects, column_x, column_y, facecolor='r')
    pl.scatter(df[column_x], df[column_y])

    figure2_window.mainloop()
#pl.show()

In [25]:
# again we build an anonymized dataset from the l-diverse partitions
dfl = build_anonymized_dataset(df, finished_l_diverse_partitions, feature_columns, sensitive_column)

Finished 1 partitions...
Finished 101 partitions...
Finished 201 partitions...
Finished 301 partitions...


In [26]:
# Let's see how l-diversity improves the anonymity of our dataset
dfl.sort_values([column_x, column_y, sensitive_column])

,age,count,education-num,income
0,17.706107,785,7.248092,<=50k
1,17.706107,1,7.248092,>50k
114,18.341463,40,3.365854,<=50k
115,18.341463,1,3.365854,>50k
4,19.320276,1301,10.000000,<=50k
5,19.320276,1,10.000000,>50k
2,20.080607,1707,9.000000,<=50k
3,20.080607,5,9.000000,>50k
122,20.364583,95,7.333333,<=50k
123,20.364583,1,7.333333,>50k


# Implementing t-closeness

As we can see, for regions where the value diversity is low, our l-diverse method produces partitions that contain a very large number of entries for one value of the sensitive attribute and only one entry for the other value. This is not ideal as while there is "plausible deniability" for a person in the dataset (after all the person could be the one "outlier") but an adversary can still be very certain about the person's attribute value in that case.

t-closeness solves this problem by making sure that the distribution of sensitive attribute values in a given partition is similar to the distribution of the values in the overall dataset.

**Implement a version of the `is_valid` function that returns `True` if the partition is diverse enough and `False` otherwise. To measure diversity, calculate the Kolmogorov-Smirnov distance between the empirical
probability distribution of the sensitive attribute over the entire dataset vs. the distribution over
the partition. Hint: the Kolmogorov-Smirnov distance is the maximum distance between the two distributions.
You can assume that the sensitive attribute is a categorical value.**

In [27]:
# here we generate the global frequencies for the sensitive column 
global_freqs = {}
total_count = float(len(df))
group_counts = df.groupby(sensitive_column)[sensitive_column].agg('count')
for value, count in group_counts.to_dict().items():
    p = count/total_count
    global_freqs[value] = p

In [28]:
global_freqs

{'<=50k': 0.7607182343065395, '>50k': 0.23928176569346055}

In [29]:
def t_closeness(df, partition, column, global_freqs):
    total_count = float(len(partition))
    d_max = None
    group_counts = df.loc[partition].groupby(column)[column].agg('count')
    for value, count in group_counts.to_dict().items():
        p = count/total_count
        d = abs(p-global_freqs[value])
        if d_max is None or d > d_max:
            d_max = d
    return d_max


def is_t_close(df, partition, sensitive_column, global_freqs, p=0.2):
    """
    :param               df: The dataframe for which to check l-diversity
    :param        partition: The partition of the dataframe on which to check l-diversity
    :param sensitive_column: The name of the sensitive column
    :param     global_freqs: The global frequencies of the sensitive attribute values
    :param                p: The maximum allowed Kolmogorov-Smirnov distance
    """
    if not sensitive_column in categorical:
        raise ValueError("this method only works for categorical values")
    return t_closeness(df, partition, sensitive_column, global_freqs) <= p

In [30]:
# Let's apply this to our dataset
finished_t_close_partitions = partition_dataset(df, feature_columns, sensitive_column, full_spans, lambda *args: is_k_anonymous(*args) and is_t_close(*args, global_freqs))

In [31]:
len(finished_t_close_partitions)

114

In [32]:
dft = build_anonymized_dataset(df, finished_t_close_partitions, feature_columns, sensitive_column)

Finished 1 partitions...
Finished 101 partitions...


In [33]:
# Let's see how t-closeness fares
dft.sort_values([column_x, column_y, sensitive_column])

,age,count,education-num,income
12,24.543287,738,11.476788,<=50k
13,24.543287,59,11.476788,>50k
2,25.747108,5617,10.000000,<=50k
3,25.747108,520,10.000000,>50k
0,26.697666,10248,8.124394,<=50k
1,26.697666,677,8.124394,>50k
26,29.000000,112,11.367647,<=50k
27,29.000000,24,11.367647,>50k
4,29.434809,3385,13.299485,<=50k
5,29.434809,1470,13.299485,>50k


In [34]:
column_x, column_y = feature_columns[:2]
t_close_rects = get_partition_rects(df, finished_t_close_partitions, column_x, column_y, indexes, offsets=[0.0, 0.0])

In [35]:
def DrawtCloseness():
    figure3_window = tk.Tk()
    figure1_window.geometry('1200x700')
    figure1_window.title('T-CLOSENESS')
    
    figure3 = pl.figure(figsize=(17,10))
    ax = pl.subplot(111)
    
    graph3 = FigureCanvasTkAgg(figure3, figure3_window)
    graph3.get_tk_widget().pack(side=tk.LEFT, fill = tk.BOTH)
    
    plot_rects(df, ax, t_close_rects, column_x, column_y, edgecolor='b', facecolor='b')
    pl.scatter(df[column_x], df[column_y])
    
    figure3_window.mainloop()
    #pl.show()

In [40]:
#validate username and password
def Validate():
    username = UsernameEntry.get()
    password = PasswordEntry.get()
    if username == 'your' and password == 'mom':
        login_window.destroy()
        
        #create new window to display landing page
        main_window = tk.Tk()
        main_window.title('PRIVACY PROTECTION IN SOCIAL NETWORKS')
        main_window.configure(bg='#DEDDE5',padx=10,pady=10)
        main_window.geometry('1400x600')
        main_window.geometry("+{}+{}".format(positionRight-650, positionDown-230))
        main_window.resizable(False, False)
        
        SampleData_label = tk.Label(main_window, bg='#DEDDE5', font = ('times', 20, 'bold'), text = "SAMPLE DATA 20x16")
        SampleData_label.grid(row = 0, column = 0, padx = 12, pady = (0, 10))
        
        # Show Data set     
        Dataset_Frame = tk.Frame(main_window)
        Dataset_Frame.grid(row=1, column=0)
        
        sf = ScrolledFrame(Dataset_Frame, width=1000, height=480)
        sf.grid(row=0,column=0)

        sf.bind_arrow_keys(Dataset_Frame)
        sf.bind_scroll_wheel(Dataset_Frame)

        inner_frame = sf.display_widget(tk.Frame)

        num_rows = 11
        num_cols = 16
        
        age = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "AGE")
        age.grid(column=0, row=0, padx=5, pady=5)
        ttk.Separator(inner_frame, orient=VERTICAL).grid(column=1, row=0, sticky='ns')
        
        workclass = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "WORKCLASS")
        workclass.grid(column=2, row=0, padx=5, pady=5)
        ttk.Separator(inner_frame, orient=VERTICAL).grid(column=3, row=0, sticky='ns')
        
        fnlwgt = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "FNLWGT")
        fnlwgt.grid(column=4, row=0, padx=5, pady=5)
        ttk.Separator(inner_frame, orient=VERTICAL).grid(column=5, row=0, sticky='ns')
        
        education = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "EDUCATION")
        education.grid(column=6, row=0, padx=5, pady=5)
        ttk.Separator(inner_frame, orient=VERTICAL).grid(column=7, row=0, sticky='ns')
        
        enum = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "NUM")
        enum.grid(column=8, row=0, padx=5, pady=5)
        ttk.Separator(inner_frame, orient=VERTICAL).grid(column=9, row=0, sticky='ns')
        
        marital = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "STATUS")
        marital.grid(column=10, row=0, padx=5, pady=5)
        ttk.Separator(inner_frame, orient=VERTICAL).grid(column=11, row=0, sticky='ns')
        
        occupation = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "OCCUPATION")
        occupation.grid(column=12, row=0, padx=5, pady=5)
        ttk.Separator(inner_frame, orient=VERTICAL).grid(column=13, row=0, sticky='ns')
        
        relationship = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "RELATIONSHIP")
        relationship.grid(column=14, row=0, padx=5, pady=5)
        ttk.Separator(inner_frame, orient=VERTICAL).grid(column=15, row=0, sticky='ns')
        
        race = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "RACE")
        race.grid(column=16, row=0, padx=5, pady=5)
        ttk.Separator(inner_frame, orient=VERTICAL).grid(column=17, row=0, sticky='ns')
        
        sex = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "SEX")
        sex.grid(column=18, row=0, padx=5, pady=5)
        ttk.Separator(inner_frame, orient=VERTICAL).grid(column=19, row=0, sticky='ns')
        
        capital_gain = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "CAPITAL-GAIN")
        capital_gain.grid(column=20, row=0, padx=5, pady=5)
        ttk.Separator(inner_frame, orient=VERTICAL).grid(column=21, row=0, sticky='ns')
        
        capital_loss = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "CAPITAL-LOSS")
        capital_loss.grid(column=22, row=0, padx=5, pady=5)
        ttk.Separator(inner_frame, orient=VERTICAL).grid(column=23, row=0, sticky='ns')
        
        hours_per_week = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "HOURS/WEEK")
        hours_per_week.grid(column=24, row=0, padx=5, pady=5)
        ttk.Separator(inner_frame, orient=VERTICAL).grid(column=25, row=0, sticky='ns')
        
        native_country = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "COUNTRY")
        native_country.grid(column=26, row=0, padx=5, pady=5)
        ttk.Separator(inner_frame, orient=VERTICAL).grid(column=27, row=0, sticky='ns')
        
        income = tk.Label(inner_frame, bg='#e7e7e7', width=11, font=('times',12,'bold'), text = "INCOME")
        income.grid(column=28, row=0, padx=5, pady=5)
        
        Data_set = "data/k-anonymity/sample_dataset.csv"
        with open(Data_set, newline="") as file:
            reader = csv.reader(file)
            r = 1
            for col in reader:
                c = 0
                for row in col:
                    label = tk.Label(inner_frame, width=11, font=('times', 12), text=row)
                    label.grid(row=r, column=(c*2), padx=5, pady=5)
                    if(c*2+1 != 7):
                        ttk.Separator(inner_frame, orient=tk.VERTICAL).grid(column=(c*2)+1, row=r, sticky='ns')
                    c += 1
                r += 1
        
        ttk.Separator(main_window, orient=VERTICAL).grid(column=1, row=0, rowspan=2, padx=(20,0), sticky='ns')
        
        frame2 = tk.Frame(main_window, relief="groove", bg='#DEDDE5')
        frame2.grid(row=0, rowspan=2,column=2,padx=10,pady=10)
        
        Button1 = tk.Button(frame2, text="K-ANONYMISITY", command=DrawkAnonymity, width=17, activebackground='#999999', bg='#8894C3', borderwidth=1, font=('times', 12))
        Button1.grid(row=0, column=2, padx=20, pady=(40,10))
        
        Label1 = tk.Label(frame2, bg='#DEDDE5', text="Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam vel nisl sed augue eleifend sagittis ut id justo.", wraplength=300, justify="left")
        Label1.grid(row=1, column=2, padx=21, pady=(0,45))
        
        Button2 = tk.Button(frame2, text="L-DIVERSIFIED", command=DrawlDiversity, width=17, activebackground='#999999', bg='#8894C3', borderwidth=1, font=('times', 12))
        Button2.grid(row=2, column=2, padx=20, pady=(20,10))
        
        Label2 = tk.Label(frame2, bg='#DEDDE5', text="Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam vel nisl sed augue eleifend sagittis ut id justo.", wraplength=300, justify="left")
        Label2.grid(row=3, column=2, padx=21, pady=(0,45))
        
        Button3 = tk.Button(frame2, text="T-CLOSENESS", command=DrawtCloseness, width=17, activebackground='#999999', bg='#8894C3', borderwidth=1, font=('times', 12))
        Button3.grid(row=4, column=2, padx=20, pady=(20,10))
        
        Label3 = tk.Label(frame2, bg='#DEDDE5', text="Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam vel nisl sed augue eleifend sagittis ut id justo.", wraplength=300, justify="left")
        Label3.grid(row=5, column=2, padx=21, pady=(0,35))
        
        main_window.mainloop()
        
    else:
        UsernameEntry.delete(first=0, last=22)
        PasswordEntry.delete(first=0, last=22)
        messagebox.showerror( message='Incorrect username or password!', icon='error')
        

# main window - admin login
login_window = tk.Tk()

windowWidth = login_window.winfo_reqwidth()
windowHeight = login_window.winfo_reqheight()
positionRight = int(login_window.winfo_screenwidth()/2 - windowWidth/2)
positionDown = int(login_window.winfo_screenheight()/2 - windowHeight/2)

login_window.title('ADMIN LOGIN')
login_window.configure(bg='#DEDDE5', padx = 40, pady = 20)
login_window.geometry('400x300')
login_window.geometry("+{}+{}".format(positionRight-300, positionDown-100))

#username label and entry
UsernameLabel = tk.Label(login_window, bg='#DEDDE5', text="Enter username", font=('times', 15))
UsernameLabel.grid(row=0, column=0, sticky=('S','W'), padx=40, pady=(10,0))

UsernameEntry = tk.Entry(login_window, width=20, font=('times', 18))
UsernameEntry.grid(row=1, column=0, padx=(40,0), pady=(5,5))

#password label and entry
PasswordLabel = tk.Label(login_window, bg='#DEDDE5', text="Enter password", font=('times', 15))
PasswordLabel.grid(row=2, column=0, sticky=('S','W'), padx=40, pady=(10,0))

PasswordEntry = tk.Entry(login_window, width=20,show="*",  font=('times', 18))
PasswordEntry.grid(row=3,column=0,padx=(40,0),pady=(5,10))

LoginButton = tk.Button(login_window, text="Login", command=Validate, width=13, activebackground='#999999', bg='#8894C3', borderwidth=1, font=('times', 15))
LoginButton.grid(row=4, column=0, padx=20, pady=20)

login_window.mainloop()